In [1]:
import os, sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import string
from glob import glob
from sklearn.utils import shuffle
%matplotlib inline

slim = tf.contrib.slim
print(tf.__version__)


/Users/sw/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/Users/sw/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


1.10.1


In [2]:
df = pd.read_csv('./dataset/review_data.csv', sep=',', names=["review","rate","emotion"])
df
new_df = df.dropna()

# new_df = shuffle(new_df)
# num_train = int(0.8 * len(new_df))

# train_df = new_df[:num_train]

# test_df = new_df[num_train:]

# len(train_df), len(test_df)

new_df["reivew_split"] = new_df["review"].str.split()
# new_df
# type(new_df["reivew_split"])

/Users/sw/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [3]:
new_df = new_df[:10000]

In [4]:
emotion_class = [
    "POSI" if new_df.iloc[i]['rate'] >= 8
    else
    "SOSO" if new_df.iloc[i]['rate'] >= 4
    else
    "NEGA"
    for i in range(new_df.shape[0])
]
new_df["class"] = emotion_class

In [5]:
new_df

,review,rate,emotion,reivew_split,class
0,종합 평점 은 4 점 드리다,4,1,"[종합, 평점, 은, 4, 점, 드리다]",SOSO
1,원작 이 칭송 받다 이유 는 웹툰 계 자체 의 질적 저하 가 심각하다 때문 원작 이...,1,0,"[원작, 이, 칭송, 받다, 이유, 는, 웹툰, 계, 자체, 의, 질적, 저하, 가...",NEGA
2,나름 의 감동 도 있다 안타깝다 마음 에 가슴 도 먹다 먹다 배우 들 의 연기 가 ...,10,2,"[나름, 의, 감동, 도, 있다, 안타깝다, 마음, 에, 가슴, 도, 먹다, 먹다,...",POSI
3,이런 걸 돈 주다 보다 내 자신 이 후회 스럽다 최악 의 쓰레기 영화 김수현 밖 에...,1,0,"[이런, 걸, 돈, 주다, 보다, 내, 자신, 이, 후회, 스럽다, 최악, 의, 쓰...",NEGA
4,초반 엔 코미디 후반 엔 액션 결론 은 코미디,7,2,"[초반, 엔, 코미디, 후반, 엔, 액션, 결론, 은, 코미디]",SOSO
5,김수현 은 멋있다 일 을 처리 하다 급하다 처리 하다 말고 절차 를 자다 밟다 하다,5,1,"[김수현, 은, 멋있다, 일, 을, 처리, 하다, 급하다, 처리, 하다, 말고, 절...",SOSO
6,원작 어디 에 팔 아 먹다,1,0,"[원작, 어디, 에, 팔, 아, 먹다]",NEGA
7,나름 웃기다 장면 도 있다 지루하다 해도 나름 볼 만 하다,6,1,"[나름, 웃기다, 장면, 도, 있다, 지루하다, 해도, 나름, 볼, 만, 하다]",SOSO
8,님 들 이영화 만들다 보다 하다 분들 묶다 두다 클레멘타인 보여주다 야하다,1,0,"[님, 들, 이영화, 만들다, 보다, 하다, 분들, 묶다, 두다, 클레멘타인, 보여...",NEGA
9,재밌다 좀 억 지 감동 짜내다 좀 유치하다,7,2,"[재밌다, 좀, 억, 지, 감동, 짜내다, 좀, 유치하다]",SOSO


In [6]:
type(new_df["reivew_split"][0])

list

In [7]:
from gensim.models.doc2vec import Doc2Vec

In [8]:
from collections import namedtuple

train_df = new_df[:int(len(new_df)*0.8)]
test_df = new_df[int(len(new_df)*0.8):]

TaggedDocument = namedtuple('TaggedDocument', 'words tags')
tagged_train_docs = [TaggedDocument(d, c) for d, c in train_df[['reivew_split', 'class']].values]
tagged_test_docs = [TaggedDocument(d, c) for d, c in test_df[['reivew_split', 'class']].values]


In [9]:
import multiprocessing
cores = multiprocessing.cpu_count()

In [10]:
doc_vectorizer = Doc2Vec(
    dm=0,            # PV-DBOW / default 1
    dbow_words=1,    # w2v simultaneous with DBOW d2v / default 0
    window=8,        # distance between the predicted word and context words
    size=300,        # vector size
    alpha=0.025,     # learning-rate
    seed=1234,
    min_count=20,    # ignore with freq lower
    min_alpha=0.025, # min learning-rate
    workers=cores,   # multi cpu
    hs = 1,          # hierarchical softmax / default 0
    negative = 10,
)

/Users/sw/anaconda3/lib/python3.6/site-packages/gensim/models/doc2vec.py:570: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


In [11]:
doc_vectorizer.build_vocab(tagged_train_docs)

In [12]:
doc_vectorizer.corpus_count
doc_vectorizer.iter

/Users/sw/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  


5

In [13]:
for epoch in range(10):
    doc_vectorizer.train(tagged_train_docs, total_examples=doc_vectorizer.corpus_count, epochs=doc_vectorizer.iter)
    doc_vectorizer.alpha -= 0.002 # decrease the learning rate
    doc_vectorizer.min_alpha = doc_vectorizer.alpha # fix the learning rate, no decay

/Users/sw/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  


In [14]:
model_name = 'Doc2vec(dbow+w,d300,n10,hs,w8,mc20,s0.001,t24).model'
doc_vectorizer.save(model_name)

In [15]:
# Load model
doc_vectorizer = Doc2Vec.load(model_name)

In [16]:
print(doc_vectorizer.wv.most_similar('쓰레기'))

[('시나리오', 0.24020668864250183), ('수준', 0.22008536756038666), ('쓰다', 0.21472547948360443), ('만하', 0.2108696848154068), ('놈', 0.20538273453712463), ('OO', 0.2032349407672882), ('질', 0.19498983025550842), ('내', 0.1944912075996399), ('스럽다', 0.19208800792694092), ('필요하다', 0.190217986702919)]


/Users/sw/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [17]:
X_train = [doc_vectorizer.infer_vector(doc.words) for doc in tagged_train_docs]
y_train = [doc.tags for doc in tagged_train_docs]

X_test = [doc_vectorizer.infer_vector(doc.words) for doc in tagged_test_docs]
y_test = [doc.tags for doc in tagged_test_docs]

In [18]:
len(X_train[0])

300

In [19]:
y_train_np = np.asarray([0 if c == 'NEGA' else 1 if c == 'SOSO' else 2 for c in y_train], dtype=int)
y_test_np = np.asarray([0 if c == 'NEGA' else 1 if c == 'SOSO' else 2 for c in y_test], dtype=int)

X_train_np = np.asarray(X_train)
X_test_np = np.array(X_test)

In [20]:
X_train_np.shape

(8000, 300)

In [21]:
y_train_np = np.eye(3)[y_train_np.reshape(-1)]
y_test_np = np.eye(3)[y_test_np.reshape(-1)]

In [22]:
import tensorflow as tf
import random

In [23]:
tf.reset_default_graph()


In [24]:
#  hyper Parameter
learning_rate = 0.001
training_epochs = 15
batch_size = 100
# input layer
X = tf.placeholder(tf.float32, [None, 300])
Y = tf.placeholder(tf.float32, [None, 3])

# dropout
keep_prob = tf.placeholder(tf.float32)

# Xavier_Initializer
xavier_init = tf.contrib.layers.xavier_initializer()
# Hidden layers and Output layer
W1 = tf.get_variable("W1", shape=[300, 256], initializer=xavier_init)
b1 = tf.Variable(tf.random_normal([256]))
L1 = tf.nn.relu(tf.matmul(X, W1)+b1)
dropout1 = tf.nn.dropout(L1, keep_prob=keep_prob)

W2 = tf.get_variable("W2", shape=[256, 256], initializer=xavier_init)
b2 = tf.Variable(tf.random_normal([256]))
L2 = tf.nn.relu(tf.matmul(dropout1, W2)+b2)
dropout2 = tf.nn.dropout(L2, keep_prob=keep_prob)

W3 = tf.get_variable("W3", shape=[256, 256], initializer=xavier_init)
b3 = tf.Variable(tf.random_normal([256]))
L3 = tf.nn.relu(tf.matmul(dropout2, W3)+b3)
dropout3 = tf.nn.dropout(L3, keep_prob=keep_prob)

W4 = tf.get_variable("W4", shape=[256, 256], initializer=xavier_init)
b4 = tf.Variable(tf.random_normal([256]))
L4 = tf.nn.relu(tf.matmul(dropout3, W4)+b4)
dropout4 = tf.nn.dropout(L4, keep_prob=keep_prob)

W5 = tf.get_variable("W5", shape=[256, 3], initializer=xavier_init)
b5 = tf.Variable(tf.random_normal([3]))
hypothesis = tf.matmul(dropout4, W5)+b5
# define cost/loss & optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=hypothesis, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Initialize
sess = tf.Session()
sess.run(tf.global_variables_initializer())
# Train Model
for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(len(X_train_np) / batch_size)

    for i in range(0, len(X_train_np), batch_size):
        batch_xs = X_train_np[i:i+batch_size]
        batch_ys = y_train_np[i:i+batch_size]

        feed_dict = {X:batch_xs, Y: batch_ys, keep_prob:0.7}
        c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
        avg_cost += c / total_batch

    print('Epoch:', '{:04d}'.format(epoch +1), 'cost =', '{:.9f}'.format(avg_cost))

print('Training Finished')

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

Epoch: 0001 cost = 1.142638013
Epoch: 0002 cost = 1.017888701
Epoch: 0003 cost = 0.979197161
Epoch: 0004 cost = 0.959993134
Epoch: 0005 cost = 0.953582252
Epoch: 0006 cost = 0.943074397
Epoch: 0007 cost = 0.930224253
Epoch: 0008 cost = 0.893113460
Epoch: 0009 cost = 0.863716896
Epoch: 0010 cost = 0.849476915
Epoch: 0011 cost = 0.832447552
Epoch: 0012 cost = 0.830971935
Epoch: 0013 cost = 0.820815646
Epoch: 0014 cost = 0.809183711
Epoch: 0015 cost = 0.774672791
Training Finished


In [26]:
correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

print('Accuracy:', sess.run(accuracy, feed_dict= {X:X_test_np, Y:y_test_np}))

InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder_2' with dtype float
	 [[Node: Placeholder_2 = Placeholder[dtype=DT_FLOAT, shape=<unknown>, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'Placeholder_2', defined at:
  File "/Users/sw/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/sw/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/sw/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/sw/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/sw/anaconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 486, in start
    self.io_loop.start()
  File "/Users/sw/anaconda3/lib/python3.6/site-packages/tornado/platform/asyncio.py", line 127, in start
    self.asynci